In [1]:
import pandas as pd
import numpy as np

# === STEP 1: LOAD DATASETS ===
try:
    wells_df = pd.read_csv("spe_africa_dseats_datathon_2025_wells_dataset.csv")
    reservoir_df = pd.read_csv("reservoir_info.csv")
    print("✅ Files loaded successfully.")
except FileNotFoundError:
    raise FileNotFoundError("❌ Check that the CSV files are in the working directory.")

# === STEP 1A: CLEAN RESERVOIR DATA ===
num_cols_reservoir = [
    'Initial Reservoir Pressure (PSI)',
    'Bubble Point Pressure (PSI)',
    'Current Average Reservoir Pressure (PSI)',
    'Solution Gas-Oil-Ratio (SCF/BBL)',
    'Formation Volume Factor (RB/STB)'
]

for col in num_cols_reservoir:
    reservoir_df[col] = (
        reservoir_df[col]
        .astype(str)
        .str.replace(",", "")
        .str.replace('"', '')
        .replace("NA", np.nan)
        .astype(float)
    )

# === STEP 1B: CLEAN WELLS DATA ===
num_cols_wells = [
    'BOTTOMHOLE_FLOWING_PRESSURE (PSI)',
    'DOWNHOLE_TEMPERATURE (deg F)',
    'ANNULUS_PRESS (PSI)',
    'CHOKE_SIZE (%)',
    'WELL_HEAD_PRESSURE (PSI)',
    'WELL_HEAD_TEMPERATURE (deg F)',
    'CUMULATIVE_OIL_PROD (STB)',
    'CUMULATIVE_FORMATION_GAS_PROD (MSCF)',
    'CUMULATIVE_TOTAL_GAS_PROD (MSCF)',
    'CUMULATIVE_WATER_PROD (BBL)',
    'ON_STREAM_HRS'
]

for col in num_cols_wells:
    wells_df[col] = (
        wells_df[col]
        .astype(str)
        .str.replace(",", "")
        .str.replace('"', '')
        .replace("NA", np.nan)
        .astype(float)
    )

# === STEP 1C: FIX DATE PARSING ===
# Check a few date examples first:
sample_dates = wells_df["PROD_DATE"].dropna().astype(str).head(5).tolist()
print("🔍 Sample PROD_DATE entries to detect format:", sample_dates)

# Update this to match your actual CSV date format
# Try %Y-%m-%d or %d/%m/%Y or %m/%d/%Y
date_format = "%Y-%m-%d"

wells_df["PROD_DATE"] = pd.to_datetime(wells_df["PROD_DATE"], format=date_format, errors="coerce")

# === STEP 1D: SELECT FIRST 20 UNIQUE WELLS ===
well_names_20 = wells_df["WELL_NAME"].dropna().unique()[:20]
wells_df_20 = wells_df[wells_df["WELL_NAME"].isin(well_names_20)]

# === PREVIEW ===
print(f"\n✅ Step 1 Complete. Total Wells Analyzed: {len(well_names_20)}")
print("🧪 Preview of cleaned data (first 5 rows):")
print(wells_df_20[["WELL_NAME", "PROD_DATE"] + num_cols_wells].head())

✅ Files loaded successfully.
🔍 Sample PROD_DATE entries to detect format: ['15-Feb-14', '16-Feb-14', '17-Feb-14', '18-Feb-14', '19-Feb-14']

✅ Step 1 Complete. Total Wells Analyzed: 20
🧪 Preview of cleaned data (first 5 rows):
  WELL_NAME PROD_DATE  BOTTOMHOLE_FLOWING_PRESSURE (PSI)  \
0   Well_#1       NaT                             4050.0   
1   Well_#1       NaT                             3961.0   
2   Well_#1       NaT                             3961.0   
3   Well_#1       NaT                             3964.0   
4   Well_#1       NaT                             3965.0   

   DOWNHOLE_TEMPERATURE (deg F)  ANNULUS_PRESS (PSI)  CHOKE_SIZE (%)  \
0                       189.866                  0.0         1.17951   
1                       189.945                  0.0         2.99440   
2                       190.004                  0.0         1.90349   
3                       190.020                  0.0         0.00000   
4                       190.107                  0.0

In [5]:
import numpy as np
import pandas as pd
from scipy.stats import linregress

# Filter: Only wells with valid (non-zero) oil & gas production
valid_gor_wells = []

for well in well_names_20:
    well_data = wells_df[wells_df["WELL_NAME"] == well].copy()

    # Keep only records with non-zero oil and gas
    valid_data = well_data[
        (well_data["CUMULATIVE_OIL_PROD (STB)"] > 0) &
        (well_data["CUMULATIVE_TOTAL_GAS_PROD (MSCF)"] > 0)
    ].copy()

    if len(valid_data) >= 5:  # At least 5 records to detect a trend
        valid_gor_wells.append(well)

# Trend Analysis for each valid well
gor_trends = []

for well in valid_gor_wells:
    well_data = wells_df[wells_df["WELL_NAME"] == well].copy()

    # Ensure proper sorting by date
    well_data = well_data.sort_values("PROD_DATE")

    # Calculate GOR
    well_data["GOR"] = (
        well_data["CUMULATIVE_TOTAL_GAS_PROD (MSCF)"] /
        well_data["CUMULATIVE_OIL_PROD (STB)"]
    )

    # Filter again for non-zero GOR
    well_data = well_data[well_data["GOR"] > 0]

    if len(well_data) < 5:
        trend = "Insufficient Data"
    else:
        # Perform linear regression on GOR vs time
        well_data["Days"] = (well_data["PROD_DATE"] - well_data["PROD_DATE"].min()).dt.days
        slope, intercept, r_value, p_value, std_err = linregress(
            well_data["Days"], well_data["GOR"]
        )

        # Define trend based on slope
        if slope > 0.1:
            trend = "Increasing"
        elif slope < -0.1:
            trend = "Decreasing"
        else:
            trend = "Stable"

    gor_trends.append({
        "WELL_NAME": well,
        "GOR Trend": trend
    })

# Compile final results
gor_trend_df = pd.DataFrame(gor_trends)

# Display Results
print("Step 4B: GOR Trend Classification (Valid Wells Only)")
print(gor_trend_df)

Step 4B: GOR Trend Classification (Valid Wells Only)
   WELL_NAME GOR Trend
0    Well_#1    Stable
1    Well_#2    Stable
2    Well_#3    Stable
3    Well_#4    Stable
4    Well_#5    Stable
5    Well_#6    Stable
6    Well_#7    Stable
7    Well_#8    Stable
8    Well_#9    Stable
9   Well_#10    Stable
10  Well_#11    Stable
11  Well_#12    Stable
12  Well_#13    Stable
13  Well_#14    Stable
14  Well_#15    Stable
15  Well_#16    Stable
16  Well_#17    Stable
17  Well_#18    Stable
18  Well_#19    Stable
19  Well_#20    Stable
